# Classification Model for Churn Analysis
- Author: Ronny Kabiru
- Program: DS-FT08 Westlands Hybrid
- Date: 07/06/2024
- TM: Maryann Mwikali


## Business Problem

SyriaTel is a telecommunications company that is concerned with the amount of customers that are leaving their service i.e. the churn. They have provided a dataset of their most recent data which has 14.5% of the customers leaving during the time period captured in the dataset. I have been tasked with analyzing the data and looking for areas where Churn is significant, and make recommendations as to what SyriaTel can do to greatly reduce the rate of churn among its customers.